In [ ]:
# --- Setup ---
!pip install -q sentence-transformers torchvision tqdm pandas scikit-learn

import os, io, requests
import pandas as pd
import numpy as np
import torch
from PIL import Image
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer
from torchvision import models, transforms
from sklearn.preprocessing import StandardScaler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.6 MB/s eta 0:00:00


In [ ]:
import os, io, requests
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from torchvision import models, transforms
from sklearn.preprocessing import StandardScaler

# --- Device setup ---
device = (
    torch.device("cuda")
    if torch.cuda.is_available()
    else torch.device("mps")
    if torch.backends.mps.is_available()
    else torch.device("cpu")
)
print(f"Using device: {device}")

# --- Constants & I/O ---
INPUT_CSV    = 'new_data.csv'
OUTPUT_CSV   = 'new_processed.csv'
IMG_CACHE    = './_imgcache'
os.makedirs(IMG_CACHE, exist_ok=True)

# --- Load & clean ---
df = pd.read_csv(INPUT_CSV, encoding='utf-8').fillna('')
num_cols = [f'feat_{i}' for i in range(43)]
df[num_cols] = df[num_cols].replace('', np.nan)
df = df.dropna(subset=['feat_0','Album_Cover_Art','Artist_Image_Link'])
df[num_cols] = StandardScaler().fit_transform(df[num_cols])

# --- Models & transforms ---
text_model = SentenceTransformer('all-MiniLM-L6-v2', device=str(device))
base_cnn   = models.resnet50(pretrained=True)
img_model  = torch.nn.Sequential(*list(base_cnn.children())[:-1]).to(device).eval()
img_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# --- Batch text embeddings ---
text_cols = ['Artist_Genre','Last_FM_Tags','Lyrics','Ground_Truth_Genre']
text_embs = {}
for col in text_cols:
    texts = df[col].tolist()
    embs = text_model.encode(
        texts,
        batch_size=64,
        convert_to_numpy=True,
        device=str(device),
        show_progress_bar=True
    )
    text_embs[col] = embs  # shape: (n_rows, emb_dim)

# --- Image dataset for fast loading + caching ---
class ImageDataset(Dataset):
    def __init__(self, urls, cache_dir, transform, prefix):
        self.urls      = urls
        self.cache_dir = cache_dir
        self.tf        = transform
        self.pref      = prefix

    def __len__(self):
        return len(self.urls)

    def __getitem__(self, idx):
        url   = self.urls[idx]
        fname = os.path.join(self.cache_dir, f"{self.pref}_{idx}.jpg")

        # if no URL or not an HTTP URL, make a blank image
        if not url or not url.lower().startswith("http"):
            img = Image.new('RGB', (224,224), color=(0,0,0))
        else:
            try:
                if os.path.exists(fname):
                    img = Image.open(fname).convert('RGB')
                else:
                    resp = requests.get(url, timeout=5)
                    img  = Image.open(io.BytesIO(resp.content)).convert('RGB')
                    img.save(fname, quality=85)
            except Exception:
                # on any download/IO error, fallback to blank
                img = Image.new('RGB', (224,224), color=(0,0,0))

        return self.tf(img)

# --- Batch image embeddings ---
image_embs = {}
for col in ['Album_Cover_Art','Artist_Image_Link']:
    urls    = df[col].tolist()
    ds      = ImageDataset(urls, IMG_CACHE, img_tf, prefix=col)
    loader  = DataLoader(ds, batch_size=64, num_workers=2, pin_memory=True)
    feats   = []
    with torch.no_grad():
        for batch in tqdm(loader, desc=f"Embed {col}"):
            batch = batch.to(device)
            out   = img_model(batch).squeeze(-1).squeeze(-1)
            feats.append(out.cpu().numpy())
    image_embs[col] = np.vstack(feats)  # (n_rows, 2048)

Using device: cuda


<ipython-input-2-09a672d3690f>:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[num_cols] = df[num_cols].replace('', np.nan)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

Batches:   0%|          | 0/208 [00:00<?, ?it/s]

Batches:   0%|          | 0/208 [00:00<?, ?it/s]

Batches:   0%|          | 0/208 [00:00<?, ?it/s]

Batches:   0%|          | 0/208 [00:00<?, ?it/s]

Embed Artist_Image_Link: 100%|██████████| 208/208 [06:11<00:00,  1.79s/it]


In [ ]:
# --- Before assembling ---
df.reset_index(drop=True, inplace=True)  # ensures i aligns with arr[i]

n_rows = len(df)
valid_mask = np.ones(n_rows, dtype=bool)

for name, arr in {**text_embs, **image_embs}.items():
    if len(arr) != n_rows:
        print(f"❌ Length mismatch in {name}: expected {n_rows}, got {len(arr)}")
        # Create a mask of valid rows based on actual array size
        tmp_mask = np.zeros(n_rows, dtype=bool)
        tmp_mask[:len(arr)] = True
        valid_mask &= tmp_mask

# --- Filter everything down to only rows with valid embeddings ---
df = df[valid_mask].reset_index(drop=True)
for k in text_embs:
    text_embs[k] = text_embs[k][valid_mask]
for k in image_embs:
    image_embs[k] = image_embs[k][valid_mask]

# --- Assemble final records ---
records = []
for i, row in tqdm(df.iterrows(), total=len(df), desc="Building records"):
    rec = {}

    # --- Text embeddings ---
    for col, arr in text_embs.items():
        for j, v in enumerate(arr[i]):
            rec[f'{col}_emb_{j}'] = v

    # --- Image embeddings ---
    for col, arr in image_embs.items():
        for j, v in enumerate(arr[i]):
            rec[f'{col}_emb_{j}'] = v

    # --- Numeric + metadata ---
    for c in num_cols:
        rec[c] = row[c]
    rec['Ground_Truth_Genre'] = row['Ground_Truth_Genre']
    rec['Release_Year']       = row['Release_Year']

    records.append(rec)

# --- Save ---
out_df = pd.DataFrame(records)
out_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Done! Saved to {OUTPUT_CSV}")

Building records: 100%|██████████| 13277/13277 [00:37<00:00, 357.17it/s]


✅ Done! Saved to new_processed.csv
